# TODO
- feature engineering sur la datetime (add year, season, weeknumber)
- add shift ( lag operator)  https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
- LSTM w/ Keras

In [1]:
%matplotlib inline
import pandas as pd, numpy as np
import sys, os

import sklearn.preprocessing as pp
import sklearn.decomposition as decomposition
import sklearn.model_selection as ms
import sklearn.metrics as metrics

import sklearn.tree as tree
import sklearn.svm as svm
import sklearn.ensemble as ensemble
import sklearn.neighbors as neighbors
# import xgboost
# import keras
sys.path.append('../')
# import lib.tools
pd.options.display.max_columns = 50

In [6]:
train_path = '../bdc/data/train/'

# Data Preprocessing

In [7]:
def season(df):
    df['season'] = df['mois']
    df['season'] = df['season'].replace([12, 1, 2], 1)
    df['season'] = df['season'].replace([3, 4, 5], 2)
    df['season'] = df['season'].replace([6, 7, 8], 3)
    df['season'] = df['season'].replace([9, 10, 11], 4)
    df['season'] = df['season'].astype('category')
    return df

In [14]:
# for file_ in os.listdir(train_path):
file_ = pd.read_csv(train_path + 'train_1.csv', header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date')

print(len(file_.columns))
#Transform to dummy
df_ddH10_rose4 = pd.get_dummies(file_['ddH10_rose4'], prefix='ddH10_rose4')
print(len(df_ddH10_rose4.columns))
df_insee = pd.get_dummies(file_.insee, prefix='insee')
print(len(df_insee.columns))
df_month = pd.get_dummies(file_.mois, prefix='month')
print(len(df_month.columns))
file_ = pd.concat([file_, df_ddH10_rose4, df_insee, df_month], axis=1)
file_.drop(['ddH10_rose4'], inplace=True, axis=1)
file_.drop(['mois'], inplace=True, axis=1)


file_.mois = file_.index.month
file_['week'] = file_.index.week
file_.insee = file_.insee.astype('str')

# Add lag operator (shift ce fait par ville)
groupby_cities = file_.groupby('insee')
shift = groupby_cities.tH2_obs.shift(1)
file_['tH2_obs_shift_1'] = shift
# file_.drop(['insee'], inplace=True, axis=1)

30
4
7
12


In [15]:
file_.describe()

,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,huH2,iwcSOL0,nbSOL0_HMoy,nH20,ntSOL0_HMoy,pMER0,rr1SOL0,...,insee_59343001,insee_67124001,insee_75114001,month_août,month_avril,month_décembre,month_février,month_janvier,month_juillet,month_juin,month_mai,month_mars,month_novembre,month_octobre,month_septembre,week,tH2_obs_shift_1
count,6167.000000,5586.000000,6.160000e+03,6048.000000,6041.000000,6160.000000,6.167000e+03,6160.000000,6.160000e+03,6160.000000,6041.000000,6160.000000,6160.000000,5845.000000,6160.000000,6027.000000,6160.000000,...,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6167.000000,6160.000000
mean,10.714788,37.366002,6.747272e-08,0.000005,3.021334,-187621.781077,-3.007864e+04,40626.427357,3.600000e-12,304.393988,82.958615,0.019749,2.194061,0.263000,4.723653,1016.782426,0.082548,...,0.142857,0.142857,0.142857,0.070375,0.102157,0.070375,0.096481,0.105562,0.070375,0.068104,0.104427,0.105562,0.068104,0.070375,0.068104,24.177072,10.710552
std,5.624477,104.122266,1.289975e-06,0.000034,1.910197,93786.779163,7.978076e+04,73618.358613,8.078591e-28,375.040197,11.797864,0.336138,2.905182,1.261355,3.317032,8.563395,0.486920,...,0.349955,0.349955,0.349955,0.255798,0.302879,0.255798,0.295274,0.307301,0.255798,0.251945,0.305838,0.307301,0.251945,0.255798,0.251945,15.202134,5.626021
min,-6.100000,0.000000,0.000000e+00,0.000000,0.008230,-503863.736917,-1.453469e+06,-224336.423922,3.600000e-12,0.466309,15.270923,0.000000,0.000000,0.000000,0.000000,982.386670,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-6.100000
25%,6.700000,0.000000,0.000000e+00,0.000000,1.736467,-254116.866438,-3.643746e+04,-2320.273261,3.600000e-12,46.977809,76.730183,0.000000,0.000000,0.000000,0.760056,1011.759314,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,6.700000
50%,10.500000,1.029342,0.000000e+00,0.000000,2.591331,-206017.425408,-5.467513e+02,21096.272421,3.600000e-12,159.982197,85.493204,0.000000,0.351207,0.000000,6.022196,1017.029292,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,10.500000
75%,14.500000,20.048593,0.000000e+00,0.000000,3.807748,-116794.746585,6.696667e+03,70471.989502,3.600000e-12,419.951200,91.789160,0.000000,4.247805,0.000000,7.995756,1022.196427,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.000000,14.500000
max,29.799999,1083.809402,4.305547e-05,0.000413,15.310204,78624.299681,9.387913e+04,641199.521351,3.600000e-12,3617.658173,99.997908,10.107500,8.000000,8.331533,8.000000,1043.259009,14.344218,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,53.000000,29.799999


In [10]:
file_.head(10)

,insee,tH2_obs,capeinsSOL0,ciwcH20,clwcH20,ffH10,flir1SOL0,fllat1SOL0,flsen1SOL0,flvis1SOL0,hcoulimSOL0,huH2,iwcSOL0,nbSOL0_HMoy,nH20,ntSOL0_HMoy,pMER0,...,insee_59343001,insee_67124001,insee_75114001,month_août,month_avril,month_décembre,month_février,month_janvier,month_juillet,month_juin,month_mai,month_mars,month_novembre,month_octobre,month_septembre,week,tH2_obs_shift_1
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-01-01,6088001,5.5,0.198757,0.0,0.000000e+00,1.970260,-386960.595711,-94260.682523,-40545.510054,3.600000e-12,66.600368,79.211738,0.0,0.000000,0.00000,4.173220,1019.543734,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,31069001,4.8,8.590036,0.0,0.000000e+00,2.755962,-200181.123871,2282.951687,14399.918192,3.600000e-12,57.954614,83.862960,0.0,0.012709,0.00000,5.736559,1018.290035,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,33281001,8.9,69.203856,0.0,1.026762e-06,3.411283,-62310.340297,9483.691316,64790.365136,3.600000e-12,202.985582,95.711708,0.0,4.792424,0.15494,7.844538,1016.139756,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,35281001,6.8,65.270316,0.0,0.000000e+00,4.655633,-61612.120075,-48356.475831,127450.659115,3.600000e-12,343.974193,90.178385,0.0,7.074669,0.00000,7.377704,1009.321502,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,59343001,5.4,5.932505,0.0,0.000000e+00,5.855930,-39553.109955,-1022.914242,117800.171480,3.600000e-12,656.972741,92.680476,0.0,7.995791,0.00000,7.995791,1008.724301,...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,67124001,2.9,0.000000,0.0,0.000000e+00,6.697897,-232184.640842,-48990.059882,190374.113081,3.600000e-12,128.078417,79.197440,0.0,0.000000,0.00000,2.729430,1015.678380,...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-01,75114001,7.0,13.271466,0.0,2.494773e-07,3.185958,-25228.272328,-18513.392636,-3158.361810,3.600000e-12,548.721209,91.507594,0.0,7.978226,0.02522,7.978226,1011.678651,...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,NaN
2014-01-02,6088001,7.9,2.588260,0.0,0.000000e+00,2.608449,-154865.268269,-123893.703248,-55913.162861,3.600000e-12,149.373482,84.961316,0.0,5.483689,0.00000,7.209474,1016.285475,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,5.5
2014-01-02,31069001,9.9,0.777403,0.0,0.000000e+00,4.282281,-140018.547252,-20709.302019,129016.229110,3.600000e-12,2017.266169,86.036733,0.0,5.099234,0.00000,8.000000,1008.707113,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,4.8


## Factor Analysis

In [15]:
fa = decomposition.FactorAnalysis()
fa.fit(file_.drop('tH2_obs'))

# ML

In [ ]:
seed = 1
results = []
names = []
predictions = [] #table of prediction for each model

In [ ]:
models = []
models.append(('CART', tree.DecisionTreeRegressor(n_jobs=3)))
models.append(('RF', ensemble.RandomForestRegressor(n_jobs=3)))
models.append(('KNN', neighbors.KNeighborsRegressor(n_jobs=3)))
models.append(('SVM', svm.SVR(n_jobs=3)))
models.append(('GB', ensemble.GradientBoostingRegressor(n_jobs=3)))
# LSTM

Apply forward chaining

In [14]:
first_train = True
for i in range(1,36):
    train_file = train_path + 'train_' + str(i) + '.csv'
    test_file = train_path + 'train_' + str(i+1) + '.csv'
    
    x_test = pd.read_csv(test_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date').drop('tH2_obs', axis=1)
    y_test = pd.read_csv(test_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date', usecols=['tH2_obs'])
    train = pd.read_csv(train_file, header=0, delimiter=';',decimal=',',
                    parse_dates=['date'], index_col='date')
    
    
    if first_train is True:
        x_train = train.drop('tH2_obs', axis=1)
        y_train = train.pop('tH2_obs')
        first_train = False
    else:
        x_train = pd.concat([x_train, train.drop('tH2_obs', axis=1)])
        y_train = pd.concat([y_train, train.pop('tH2_obs')])
        
    for name, m in models: 
    pred = ms.cross_val_predict(m, dff[features], dff[label], cv=kfold, n_jobs=3)
    cv_res = ms.cross_val_score(m, dff[features], dff[label], cv=kfold, n_jobs=3, scoring = 'accuracy')
    m.fit
    
    predictions.append(pred)
    results.append(cv_res)
    names.append(name)
    print "Score %s: %.3f%%, %.3f%%" % (name, cv_res.mean()*100, cv_res.std()*100)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


# Feature engineering

## TODO
- Travail d'une force : vitesse * direction  
https://fr.wikipedia.org/wiki/Travail_d%27une_force


# Draft

In [ ]:
train['week'] = train.index.week
train = season(train)

In [29]:
groupby_cities = file_.groupby('insee')

Lag operator

In [30]:
# test = groupby_cities.tH2_obs.shift(1)
# test.fillna(0, inplace=True)
pd.DataFrame(groupby_cities.tH2_obs.rolling(window=2).mean())

tH2_obs
insee    date                 
31069001 2014-01-01        NaN
         2014-01-02   7.350000
         2014-01-03  10.700000
         2014-01-04   9.900000
         2014-01-05   6.800000
         2014-01-06   6.950000
         2014-01-07   9.750000
         2014-01-08  11.150000
         2014-01-09  11.450000
         2014-01-10   9.550000
         2014-01-11   8.650000
         2014-01-12   9.900000
         2014-01-13   9.750000
         2014-01-14   6.000000
         2014-01-15   3.950000
         2014-01-16   6.350000
         2014-01-17   5.500000
         2014-01-18   3.450000
         2014-01-19   3.350000
         2014-01-20   4.700000
         2014-01-21   5.600000
         2014-01-22   3.300000
         2014-01-23   4.050000
         2014-01-24   7.650000
         2014-01-25   9.700000
         2014-01-26  10.550000
         2014-01-27  10.150000
         2014-01-28   6.700000
         2014-01-29   4.650000
         2014-01-30   5.100000
...                        ...
75114001 2016-05-01   7.050000
         2016-05-02   7.400000
         2016-05-03   9.950000
         2016-05-04  10.000000
         2016-05-05  10.150000
         2016-05-06  12.350000
         2016-05-07  14.300000
         2016-05-08  15.700000
         2016-05-09  16.900000
         2016-05-10  16.550000
         2016-05-11  15.050000
         2016-05-12  14.600000
         2016-05-13  14.450000
         2016-05-14  12.500000
         2016-05-15   9.050000
         2016-05-16   9.550000
         2016-05-17  10.750000
         2016-05-18  12.250000
         2016-05-19  11.900000
         2016-05-20  10.900000
         2016-05-21  13.650000
         2016-05-22  15.300000
         2016-05-23  13.450000
         2016-05-24  10.900000
         2016-05-25  10.050000
         2016-05-26  11.400000
         2016-05-27  15.000000
         2016-05-28  16.750000
         2016-05-29  16.650001
         2016-05-30  15.850000

[6167 rows x 1 columns]

In [37]:
groupby_cities.tH2_obs.head()

date
2014-01-01     5.5
2014-01-01     4.8
2014-01-01     8.9
2014-01-01     6.8
2014-01-01     5.4
2014-01-01     2.9
2014-01-01     7.0
2014-01-02     7.9
2014-01-02     9.9
2014-01-02    10.9
2014-01-02    10.4
2014-01-02     9.8
2014-01-02     7.1
2014-01-02    10.8
2014-01-03     9.1
2014-01-03    11.5
2014-01-03    10.3
2014-01-03    11.7
2014-01-03     8.9
2014-01-03     5.7
2014-01-03     9.8
2014-01-04    11.3
2014-01-04     8.3
2014-01-04     9.3
2014-01-04     9.5
2014-01-04     7.3
2014-01-04     8.2
2014-01-04     7.8
2014-01-05    11.2
2014-01-05     5.3
2014-01-05     7.6
2014-01-05     5.7
2014-01-05     5.6
2014-01-05     6.5
2014-01-05     5.9
Name: tH2_obs, dtype: float64

Interpolate NA val

In [ ]:
groupby_city.capeinsSOL0.apply(pd.Series.interpolate)